In [13]:
import pandas as pd

student = pd.DataFrame(data=[
    {'姓名':'徐國堂','國文':87,'英文':92,'數學':86},
    {'姓名':'王國堂','國文':92,'英文':80,'數學':90},
    {'姓名':'張國堂','國文':76,'英文':83,'數學':65}
])
student


,姓名,國文,英文,數學
0,徐國堂,87,92,86
1,王國堂,92,80,90
2,張國堂,76,83,65


In [14]:
import requests
#下載csv檔
csv_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
res = requests.get(url=csv_url)
if res.ok:
    print("下載成功")
else:
    print("下載失敗")

下載成功


In [15]:
import io
import csv
csv_str:str = res.text
csv_file = io.StringIO(csv_str)
dict_reader = csv.DictReader(csv_file)
csv_data = list(dict_reader)

In [16]:
from pydantic import BaseModel,Field,field_serializer,RootModel
from datetime import datetime

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    城鎮:str = Field(alias='county')
    aqi:int
    狀態:str = Field(alias='status')
    日期:datetime = Field(alias='datacreationdate')

    @field_serializer("日期")
    def serialize_str(self,value):
        return value.strftime('%a %d %b %Y, %I:%M%p')

class Csvroot(RootModel):
    root:list[Site]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

In [17]:
csv_root = Csvroot.model_validate(csv_data)
csv_root.model_dump()

[{'站點名稱': '屏東(枋山)',
  '城鎮': '屏東縣',
  'aqi': 50,
  '狀態': '良好',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '新北(樹林)',
  '城鎮': '新北市',
  'aqi': 91,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '屏東(琉球)',
  '城鎮': '屏東縣',
  'aqi': 97,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '臺南(麻豆)',
  '城鎮': '臺南市',
  'aqi': 78,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '高雄(湖內)',
  '城鎮': '高雄市',
  'aqi': 60,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '彰化(員林)',
  '城鎮': '彰化縣',
  'aqi': 80,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '大城',
  '城鎮': '彰化縣',
  'aqi': 100,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '富貴角',
  '城鎮': '新北市',
  'aqi': 58,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '麥寮',
  '城鎮': '雲林縣',
  'aqi': 92,
  '狀態': '普通',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '關山',
  '城鎮': '臺東縣',
  'aqi': 31,
  '狀態': '良好',
  '日期': 'Sun 17 Mar 2024, 03:00PM'},
 {'站點名稱': '馬公',
  '城鎮': 

In [18]:
import pandas as pd
from pandas import DataFrame
aqi:DataFrame = DataFrame(data=csv_root.model_dump())
aqi

,站點名稱,城鎮,aqi,狀態,日期
0,屏東(枋山),屏東縣,50,良好,"Sun 17 Mar 2024, 03:00PM"
1,新北(樹林),新北市,91,普通,"Sun 17 Mar 2024, 03:00PM"
2,屏東(琉球),屏東縣,97,普通,"Sun 17 Mar 2024, 03:00PM"
3,臺南(麻豆),臺南市,78,普通,"Sun 17 Mar 2024, 03:00PM"
4,高雄(湖內),高雄市,60,普通,"Sun 17 Mar 2024, 03:00PM"
...,...,...,...,...,...
995,萬華,臺北市,87,普通,"Sun 17 Mar 2024, 04:00AM"
996,中山,臺北市,79,普通,"Sun 17 Mar 2024, 04:00AM"
997,士林,臺北市,77,普通,"Sun 17 Mar 2024, 04:00AM"
998,淡水,新北市,82,普通,"Sun 17 Mar 2024, 04:00AM"
